In [8]:
# So we can load files from other sub-directories, e.g. datasets.
import os
import sys
module_path = os.path.abspath(os.path.join('unlabeled_extrapolation'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from unlabeled_extrapolation.models import imnet_resnet
from unlabeled_extrapolation.models import imnet_models
from torch import nn
import torch

import importlib
importlib.reload(imnet_resnet)
importlib.reload(imnet_models)


<module 'unlabeled_extrapolation.models.imnet_models' from '/juice/scr/ananya/cifar_experiments/unlabeled_extrapolation/unlabeled_extrapolation/models/imnet_models.py'>

In [2]:
model = imnet_resnet.ResNet50(pretrained=True, pretrain_style='mocov2', checkpoint_path='/u/scr/ananya/simclr_weights/moco_v2_800ep_pretrain.pth.tar')

# Try loading fmow checkpoints

In [8]:
model1 = imnet_models.ResNet50(pretrained=True, pretrain_style='mocov2', checkpoint_path='/u/scr/ananya/simclr_weights/moco_checkpoint_0200.pth.tar')

In [9]:
model2 = imnet_resnet.ResNet50(pretrained=True, pretrain_style='mocov2', checkpoint_path='/u/scr/ananya/simclr_weights/mocotp_checkpoint_0200.pth.tar')

In [ ]:
# Print to make sure model1 and model2 are different, so loading is actually doing something.
print(list(list(model2._model.children())[0].parameters()))

# Try to examine layers, children, parameters

In [80]:
def count_parameters(model, trainable):
    return sum(p.numel() for p in model.parameters() if p.requires_grad == trainable)

In [53]:
list(model._model.children())[-3]

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): 

In [79]:
# Tuning batch norm parameters only.
model._model.eval()
model.set_requires_grad(False)
for layer in model._model.modules():
    if isinstance(layer,nn.modules.batchnorm.BatchNorm2d): 
        for param in layer.parameters():
            param.requires_grad = True

count_parameters(model, True)

53120

In [108]:
model._model.eval()
model.set_requires_grad(False)
num_layers_fine_tune = 5
for child in list(model._model.children())[:num_layers_fine_tune]:
    for param in child.parameters():
            param.requires_grad = True
count_parameters(model, True)

hi
hi
hi
hi
hi


225344

In [137]:
def get_params(layers):
    params = []
    for layer in layers:
        for param in layer.parameters():
            params.append(param)
    return params

base_params = get_params(list(model._model.children())[:-1])
fc_params = get_params(list(model._model.children())[-1:])
len(base_params)

159

In [47]:
list(model._model.named_children())

[('conv1',
  Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)),
 ('bn1',
  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
 ('relu', ReLU(inplace=True)),
 ('maxpool',
  MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)),
 ('layer1',
  Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), str

In [134]:
swav_path = '/u/scr/ananya/simclr_weights/swav_800ep_pretrain.pth.tar'
model = imnet_resnet.ResNet50(pretrained=True, pretrain_style='swav', checkpoint_path=swav_path)

In [135]:
model

ResNet50(
  (_model): Sequential(
    (resnet): ResNet(
      (padding): ConstantPad2d(padding=(1, 1, 1, 1), value=0.0)
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(2, 2), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

In [17]:
from unlabeled_extrapolation.datasets import breeds
entity30 = breeds.Breeds(root='/u/scr/nlp/imagenet/', breeds_name='entity30', transform=trans, split='val')

In [11]:
import torchvision
import torch
import torchvision.transforms as transforms
trans = transforms.Compose(
[
    transforms.Resize(size=[224,224]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
x = torch.unsqueeze(trans(entity30[0][0]), axis=0)
x = x.cuda()
print(x.shape)
model.eval()
model.cuda()
z = model._model(x)

torch.Size([1, 3, 224, 224])


In [18]:
print(len(entity30))

6000


In [19]:
for i in range(5):
    print(i)
    loader = torch.utils.data.DataLoader(entity30, batch_size=64,shuffle=True, num_workers=2)
    with torch.no_grad():
        for data in loader:
            print('hi')
            images, labels = data
            images = images.cuda()
            outputs = model(images)


0
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
1
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
2
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
3
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi